In [1]:
import numpy as np
import pandas as pd
!pip install --user -U nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

from sklearn.metrics import accuracy_score,confusion_matrix

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
dff = pd.read_csv("spam.csv", encoding = "ISO-8859-1", usecols=[0,1], skiprows=1,names=["target", "msg"])

In [3]:
dff.head(10)

,target,msg
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [4]:
dff['target'].value_counts()

ham     4825
spam     747
Name: target, dtype: int64

In [5]:
dff.duplicated().sum()

403

In [6]:
dff.drop_duplicates(inplace=True)

In [7]:
dff['target'].value_counts()  # need to balance data before applying any model

ham     4516
spam     653
Name: target, dtype: int64

In [8]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5169 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   target  5169 non-null   object
 1   msg     5169 non-null   object
dtypes: object(2)
memory usage: 121.1+ KB


In [9]:
#def lowercasing(dff):
#    dff['mesg'] = dff['msg'].str.lower()
#    return dff['mesg']

In [10]:
dff['msg'] = dff['msg'].apply(lambda x:x.lower())
dff['msg'][2]

"free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005. text fa to 87121 to receive entry question(std txt rate)t&c's apply 08452810075over18's"

In [11]:
#dff['msg'] = dff['msg'].apply(lowercasing)

In [12]:
import re

dff['msg']  = dff['msg'].apply(lambda x: re.sub('[\d+]',' ',x))
dff['msg'][2]

"free entry in   a wkly comp to win fa cup final tkts   st may     . text fa to       to receive entry question(std txt rate)t&c's apply            over  's"

In [13]:

dff['msg']  = dff['msg'].apply(lambda x: re.sub('[\)&.(]',' ',x))
dff['msg'][2]

"free entry in   a wkly comp to win fa cup final tkts   st may       text fa to       to receive entry question std txt rate t c's apply            over  's"

In [14]:
dff['msg'] = dff['msg'].apply(lambda x: re.sub(re.compile('<.*?>'), '', x))

In [15]:
import nltk
nltk.download ('stopwords')
from nltk.corpus import stopwords


stop_words_list = stopwords.words('english')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
def removestop(df):
    sentence = df.split()
    filterdf = [word.lower() for word in sentence if word not in stop_words_list]
        
    return ' '.join(filterdf)

In [17]:
dff['msg']= dff['msg'].apply(removestop)
dff['msg'][2]

"free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate c's apply 's"

In [19]:
dff['target'] = dff['target'].map({'ham':1,'spam':0})

In [21]:
dff

,target,msg
0,1,"go jurong point, crazy available bugis n great..."
1,1,ok lar joking wif u oni
2,0,free entry wkly comp win fa cup final tkts st ...
3,1,u dun say early hor u c already say
4,1,"nah think goes usf, lives around though"
...,...,...
5567,0,nd time tried contact u u å£ pound prize claim...
5568,1,ì_ b going esplanade fr home?
5569,1,"pity, * mood suggestions?"
5570,1,guy bitching acted like i'd interested buying ...


In [22]:

dff

,target,msg
0,1,"go jurong point, crazy available bugis n great..."
1,1,ok lar joking wif u oni
2,0,free entry wkly comp win fa cup final tkts st ...
3,1,u dun say early hor u c already say
4,1,"nah think goes usf, lives around though"
...,...,...
5567,0,nd time tried contact u u å£ pound prize claim...
5568,1,ì_ b going esplanade fr home?
5569,1,"pity, * mood suggestions?"
5570,1,guy bitching acted like i'd interested buying ...


In [23]:
from nltk.tokenize import TweetTokenizer
toknzr = TweetTokenizer()


dff['msg'] = dff['msg'].apply(lambda x: toknzr.tokenize(x))
dff

,target,msg
0,1,"[go, jurong, point, ,, crazy, available, bugis..."
1,1,"[ok, lar, joking, wif, u, oni]"
2,0,"[free, entry, wkly, comp, win, fa, cup, final,..."
3,1,"[u, dun, say, early, hor, u, c, already, say]"
4,1,"[nah, think, goes, usf, ,, lives, around, though]"
...,...,...
5567,0,"[nd, time, tried, contact, u, u, å, £, pound, ..."
5568,1,"[ì_, b, going, esplanade, fr, home, ?]"
5569,1,"[pity, ,, *, mood, suggestions, ?]"
5570,1,"[guy, bitching, acted, like, i'd, interested, ..."


In [24]:
#dff['msg'] = dff['msg'].apply(lambda x: toknzr.tokenize(x))
dff['msg']=dff['msg'].astype(str)

In [25]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
bow = CountVectorizer(max_features=3000)
tfidf = TfidfVectorizer()

In [26]:
xx_bow = bow.fit_transform(dff['msg'])
#xxtest = bow.transform(xtest['col'])

In [27]:
y = dff['target']


In [28]:
xx_tfidf = tfidf.fit_transform(dff['msg'])

In [29]:
from imblearn.over_sampling import SMOTE

sampling_strategy = 0.8  
random_state = 42  
k_neighbors = 4  

smote = SMOTE(sampling_strategy=sampling_strategy, random_state=random_state, k_neighbors=k_neighbors)

xx, yy = smote.fit_resample(xx_bow, y)

In [30]:
xx.shape, yy.shape,

((8128, 3000), (8128,))

In [31]:
yy.value_counts()

1    4516
0    3612
Name: target, dtype: int64

In [32]:

from sklearn.model_selection import train_test_split

x,x_test,y,y_test = train_test_split(xx,yy,test_size = 0.2,random_state = 42)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
rf_classifier = RandomForestClassifier()

param_grids = {"n_estimators": [50,100,150,200,250],
              "max_depth": [1,3,5,7],
              "criterion": ["gini","entropy"]
              }

model = model_selection.GridSearchCV(estimator=rf_classifier,
                                    param_grid=param_grids,
                                    scoring="accuracy",
                                    verbose=2,
                                    cv=5)

model.fit(x,y)
print(model.best_score_)
print(model.best_estimator_.get_params())

In [33]:
!pip install optuna

In [35]:
###    random forest with optuna
import optuna
def objective(trial:optuna.Trial,x,y):
    n_estimators = trial.suggest_int('n_estimators',50,100,150)
    max_depth=trial.suggest_int('max_depth', 1,3,5,7)
    criterion= trial.suggest_categorical('criterion','gini','entropy')
    max_features = trial.suggest_uniform('max_feature',0.01,1.0)
              
    
    rf_classifier = RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          criterion=criterion,
                                          max_features=max_features)




    y_pred = rf_classifier.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)
    print(classification_report(y_test, y_pred))
    return accuracy
    
objective = objective(x=x,y=y,trial=15)

study = optuna.create_study(direction="minimize")
study.optimize(objective,n_trials=15)

best_params = study.best_params
best_accuracy = study.best_value

print(f"Best Hyperparameters: {best_params}")
print(f"Best Accuracy: {best_accuracy}")


AttributeError: 'int' object has no attribute 'suggest_int'

In [36]:
!pip install xgboost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

xgb_classifier = xgb.XGBClassifier()


param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],  
    'max_depth': [3, 4, 5],           
    'subsample': [0.8, 0.9, 1.0],     
    'colsample_bytree': [0.8, 0.9, 1.0], 
    'min_child_weight': [1, 3, 5],    
    'gamma': [0, 0.1, 0.2],           
    'reg_lambda': [0, 1, 2],          
    'reg_alpha': [0, 1, 2],           
}


xgb_model = GridSearchCV(estimator=xgb_classifier,
                        param_grid=param_grid,
                        scoring="accuracy")


xgb_model.fit(x,y)

print(model.best_score_)
print(model.best_estimator_.get_params())



#y_pred = classifier.predict(X_test_tfidf)
#accuracy = accuracy_score(y_test, y_pred)
#print("Accuracy:", accuracy)
#print(classification_report(y_test, y_pred))

In [ ]:
#from sklearn.ensemble import RandomForestClassifier

# Train the Random Forest classifier
#classifier = RandomForestClassifier()
#classifier.fit(X_train_tfidf, y_train)

# Predict and evaluate
#y_pred = classifier.predict(X_test_tfidf)
#accuracy = accuracy_score(y_test, y_pred)
#print("Accuracy:", accuracy)
#print(classification_report(y_test, y_pred))

In [38]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV



svm_classifier = SVC()
#classifier.fit(X_train_tfidf, y_train)
params ={'C': [0.1,1,10],
        'kernel':['linear','rbf'],
        'gamma': ['scale','auto',0.1],
        'degree': [2,3,4]}

svm_model = GridSearchCV(estimator=svm_classifier,
                        param_grid=params,
                        scoring='accuracy')

svm_model.fit(x,y)



print(svm_model.best_params_)
print(svm_model.best_score_)



# Predict and evaluate
#y_pred = classifier.predict(X_test_tfidf)
#accuracy = accuracy_score(y_test, y_pred)
#print("Accuracy:", accuracy)
#print(classification_report(y_test, y_pred))

{'C': 1, 'degree': 2, 'gamma': 0.1, 'kernel': 'rbf'}
0.9592423579495062


In [39]:
from sklearn.linear_model import LogisticRegression

logreg_model = LogisticRegression()

params = {'C': [0.001, 0.01, 0.1, 1, 10],           
    'penalty': ['l1', 'l2'],                  
    'max_iter': [100, 200, 300],              
    'solver': ['liblinear', 'saga'],          
    'multi_class': ['ovr', 'multinomial'],    
    'class_weight': [None, 'balanced']}

logi_model = GridSearchCV(estimator=logreg_model,param_grid=params,scoring='accuracy',cv=5)

logi_model.fit(x, y)


print(grid_search.best_params_)
print(grid_search.best_score_)

NameError: name 'X' is not defined

In [ ]:
from sklearn.naive_bayes import MultinomialNB


classifier = MultinomialNB()
classifier.fit(X_train_tfidf, y_train)


y_pred = classifier.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))